In [1]:
import pandas as pd
import numpy as np
from os import listdir


# Сведение файлов

In [ ]:
%%time
p = '/Users/ivansinazhenski/Documents/ WORK/ГПХ_B2C_Воронков_11_2020/Исходники/исходники_для_чтения'
files = []

for f in listdir(p):
    print(f)
    df = pd.read_excel(p+'/'+f)
    df.loc[:,'mon'] = f.split('_')[0]
    files.append(df)

In [ ]:
payments = pd.concat(files)

In [ ]:
%%time

payments.to_excel('/Users/ivansinazhenski/Documents/ WORK/ГПХ_B2C_Воронков_11_2020/Исходники/р12_все_выплаты.xlsx', index=False)

# Чтение файлов

In [2]:
%%time

payments = pd.read_excel('/Users/ivansinazhenski/Documents/ WORK/ГПХ_B2C_Воронков_11_2020/Исходники/р12_все_выплаты.xlsx')

CPU times: user 28 s, sys: 347 ms, total: 28.4 s
Wall time: 29.8 s


In [3]:
%%time

mrf_cods = pd.read_excel('/Users/ivansinazhenski/Documents/ WORK/ГПХ_B2C_Воронков_11_2020/Исходники/код_филиал.xlsx')

CPU times: user 70.7 ms, sys: 20.7 ms, total: 91.4 ms
Wall time: 154 ms


# Подготовка исходного файла к созданию сводной

In [4]:
# берем только Массовый сегмент

mass_seg = payments[(payments['Подразделение'].str.contains('Массовый сегмент'))|(payments['Подразделение'].str.contains('массовым сегмент'))].copy()

In [5]:
# выделяем код филиала

mass_seg = mass_seg.merge(mass_seg['Подразделение'].str.split('.', expand=True)[0].to_frame().rename(columns={0:'code_mrf'}),
              how='inner', left_index=True, right_index=True)

In [6]:
mrf_cods['КОД_Ф'] = mrf_cods['КОД_Ф'].astype(str)

In [7]:
mrf_cods['КОД_Ф'] = '0'+ mrf_cods['КОД_Ф']

In [8]:
#сводим код и название мрф и рф

merged = mass_seg.merge(mrf_cods, how='left', left_on='code_mrf', right_on='КОД_Ф')

In [9]:
merged[merged['ФИЛИАЛ_И'].isnull()]['code_mrf'].value_counts()

01102    1747
Name: code_mrf, dtype: int64

In [10]:
#дозаполняем несведенные

merged['ФИЛИАЛ_И'].fillna('Филиал в Брянской и Орловской областях', inplace=True)
merged['УЗЕЛ_И'].fillna('Центр', inplace=True)

## Выделение подразделений для группировки

In [11]:
# делим на части общий путь

podrazdel = merged['Подразделение'].str.split(' / ', expand=True)

In [12]:
# Заполняем последними значениями


podrazdel.loc[podrazdel[podrazdel[4].isnull()].index, 4] = podrazdel[3]
podrazdel.loc[podrazdel[podrazdel[4].isnull()].index, 4] = podrazdel[2]
podrazdel.loc[podrazdel[podrazdel[4].isnull()].index, 4] = podrazdel[1]
podrazdel.loc[podrazdel[podrazdel[4].isnull()].index, 4] = podrazdel[0]

In [13]:
# выкидываем не нужные столбцы

podrazdel = podrazdel.loc[:,[0,4]]

In [14]:
# выделяем лист значений который надо отбросить

s = '.'
to_replace = (podrazdel[0].str.split('.', expand=True)[0]+s+podrazdel[0].str.split('.', expand=True)[1]+s).drop_duplicates().to_list()
to_replace += (podrazdel[0].str.split('.', expand=True)[0]+s+'0353'+s).drop_duplicates().to_list()

In [15]:
%%time
# убираем числовые значения

for tr in to_replace:
    podrazdel[4] = podrazdel[4].str.replace(tr, '')

CPU times: user 25.5 s, sys: 210 ms, total: 25.7 s
Wall time: 27.5 s


In [ ]:
%%time 
# доубираем числа в начале и в конце

s = '.'
for i in podrazdel.index:
    print(i)
    split_4 = podrazdel.loc[i, 4].split('.')
    try:
        int(split_4[0])
        split_4 = split_4[2:]
        podrazdel.loc[i, 4] = s.join(split_4)
    except:
        pass

In [20]:
b = podrazdel[4].copy()

In [ ]:
podrazdel[4].value_counts().to_excel('a.xlsx')

In [17]:
podrazdel_v2 = pd.read_excel('a.xlsx')

In [22]:
mass_seg_to_work = merged.merge(podrazdel, how='inner', left_index=True, right_index=True)

In [23]:
mass_seg_to_work = mass_seg_to_work.merge(podrazdel_v2, how= 'inner', left_on=4, right_on=4)


In [24]:

mass_seg_to_work.rename(columns={'low':'направление_B2C',
                         'hight': 'Направление B2C верх'}, inplace=True)

In [ ]:
# готовимся ко сведению с основным df

podrazdel = podrazdel.loc[:, [4, 'Направление B2C верх']]
podrazdel.rename(columns={4:'направление_B2C'}, inplace=True)

In [ ]:
# сводим с основным df

mass_seg_to_work = merged.merge(podrazdel, how='inner', left_index=True, right_index=True)

In [ ]:
mass_seg_to_work[mass_seg_to_work['Номер назначения'] == '351884']

# Создание общей сводной таблицы (с мрф)

In [25]:
# готовим столбец для агрегирования

mass_seg_to_work['all_pivot'] = mass_seg_to_work['УЗЕЛ_И']+'|'+mass_seg_to_work['ФИЛИАЛ_И']+'|'+mass_seg_to_work['Направление B2C верх']+'|'+mass_seg_to_work['направление_B2C']+'|'+mass_seg_to_work['Номер назначения']


In [26]:
mass_seg_to_work = mass_seg_to_work[(mass_seg_to_work['mon'] != 9) & (mass_seg_to_work['mon'] != 8)].copy()

In [27]:
mass_seg_to_work.mon.value_counts()

7    17306
4    16645
5    15916
3    15835
1    15692
6    15451
2    14822
Name: mon, dtype: int64

In [28]:
# делаем сводную

summary = pd.pivot_table(mass_seg_to_work, columns=['mon'], index=['all_pivot'], values='Сумма, руб.коп.', aggfunc='sum')

In [30]:
# summary['Направление B2C'] = summary['Направление B2C'].replace('Управление по работе с массовым сегментом_101', 'Управление по работе с массовым сегментом')
# summary['Направление B2C'] = summary['Направление B2C'].replace('Управление по работе с массовым сегментом_116', 'Управление по работе с массовым сегментом')
# # summary['Направление B2C'] = summary['Направление B2C'].replace('Коммерческий блок_104', 'Коммерческий блок')
# summary['Направление B2C'] = summary['Направление B2C'].replace('Подразделения по работе с массовым сегментом.1', 'Подразделения по работе с массовым сегментом')

In [31]:
# заполняем нули

summary.fillna(0, inplace=True)

In [32]:
# выделяем столбец для разбиения

summary.reset_index(inplace=True)

In [33]:
# разбиваем столбец на элемент

all_pivot_to_join = summary['all_pivot'].str.split('|', expand=True).rename(columns={0:'МРФ', 1:'Филиала', 2:'Направление B2C_верх', 3:'Направление B2C', 4:'Номер_назначения'})
summary = summary.merge(all_pivot_to_join, how='inner', left_index=True, right_index=True)

In [34]:
# добовляем итоги по строке, метку превышений 500к

summary.loc[:,'Всего'] = summary.loc[:, [1,2,3,4,5,6,7]].sum(1)
summary.loc[summary[summary['Всего'] >= 500000].index, 'Превышение >= 500к'] = 1

In [35]:
for cl in [1,2,3,4,5,6,7,'Всего']:
    summary[cl]/= 1000

In [40]:
summary = summary.loc[:, [               'МРФ',         'Направление B2C_верх', 'Направление B2C',  'Филиала', 
         'Номер_назначения',  'Всего',                  1,                    2,
                          3,                    4,                    5,
                          6,                    7,              
       'Превышение >= 500к',   'Месяца_без_оплат',    'нет оплат 3 мес']]

In [41]:
summary.loc[:,'кол_во'] = 1

In [42]:
# считаем среднюю выплату

summary.loc[:, 'мес_оплат']= summary['Месяца_без_оплат'].str.split(',', expand=True).isnull().sum(1)
summary['Средняя_Выплата'] = summary['Всего']/summary['мес_оплат']

In [29]:
summary#.to_excel('f.xlsx')

mon,1,2,3,4,5,6,7
all_pivot,,,,,,,
Волга|Кировский филиал|Группа активных продаж|Группа активных продаж|100178-3,3311.45,1179.31,592.52,2978.16,2758.22,1061.81,1211.39
Волга|Кировский филиал|Группа активных продаж|Группа активных продаж|101084-4,4373.58,2155.18,2241.39,1551.73,344.83,2126.44,4942.55
Волга|Кировский филиал|Группа активных продаж|Группа активных продаж|101803-5,4149.99,2991.36,730.46,601.73,3518.40,3979.72,1741.97
Волга|Кировский филиал|Группа активных продаж|Группа активных продаж|102083-12,11571.81,1364.37,556.90,8418.98,12942.11,6736.43,5627.37
Волга|Кировский филиал|Группа активных продаж|Группа активных продаж|102570-2,NaN,1091.95,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
Юг|Ставропольский филиал|Договоры ГПХ ЦФО 015 Блок по работе с массовым сегментом|Договоры ГПХ ЦФО 015 Блок по работе с массовым сегментом|602857,NaN,NaN,NaN,NaN,NaN,NaN,1198.00
Юг|Ставропольский филиал|Договоры ГПХ ЦФО 015 Блок по работе с массовым сегментом|Договоры ГПХ ЦФО 015 Блок по работе с массовым сегментом|602884,NaN,NaN,NaN,NaN,NaN,NaN,1650.00
Юг|Ставропольский филиал|Договоры ГПХ ЦФО 015 Блок по работе с массовым сегментом|Договоры ГПХ ЦФО 015 Блок по работе с массовым сегментом|602931,NaN,NaN,NaN,NaN,NaN,NaN,2440.00


In [43]:
%%time

# общий итог
summary_itog = summary.copy()
summary_itog['МРФ'] = summary_itog['МРФ']+'_а1'
summary_itog['Филиала'] = summary_itog['Филиала']+'_а1'
summary_itog['Направление B2C_верх'] = summary_itog['Направление B2C_верх']+'_а1'
summary_itog['Направление B2C'] = summary_itog['Направление B2C']+'_а1'
summary_itog['level'] = 4

summary_itog.loc['1111_Итог', 'МРФ'] = '1111_Итог'
summary_itog.loc['1111_Итог', 'level'] = 0
summary_itog.loc['1111_Итог', 'Номер_назначения'] = summary_itog['Номер_назначения'].count()
summary_itog.loc['1111_Итог', 'мес_оплат'] = summary_itog['мес_оплат'].mean()

for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к', 'нет оплат 3 мес', 'кол_во']:
    summary_itog.loc['1111_Итог', cl] = summary_itog[cl].sum()
summary_itog.loc['1111_Итог', 'Средняя_Выплата'] =  summary_itog.loc['1111_Итог', 'Всего']/summary_itog.loc['1111_Итог', 'мес_оплат']

for mrf in summary_itog.МРФ.value_counts().index:
    if mrf.find('1111_Итог') == 0:
        continue
    mrf_all = summary_itog[summary_itog['МРФ'] == mrf].copy()
    
    summary_itog.loc['1111_'+mrf, 'level'] = 0
    summary_itog.loc['1111_'+mrf, 'МРФ'] = mrf.split('_')[0]
    summary_itog.loc['1111_'+mrf,'Номер_назначения'] = mrf_all['Номер_назначения'].count()
    summary_itog.loc['1111_'+mrf, 'мес_оплат'] = mrf_all['мес_оплат'].mean()
    
    for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к', 'нет оплат 3 мес', 'кол_во']:
        summary_itog.loc['1111_'+mrf, cl] = mrf_all[cl].sum()
    summary_itog.loc['1111_'+mrf, 'Средняя_Выплата'] =  summary_itog.loc['1111_'+mrf, 'Всего']/summary_itog.loc['1111_'+mrf, 'мес_оплат']
                     
        
    for b2c in mrf_all['Направление B2C_верх'].value_counts().index:
        b2c_up = mrf_all[mrf_all['Направление B2C_верх'] == b2c].copy()

        summary_itog.loc['1111_'+mrf+b2c, 'МРФ'] = mrf
        summary_itog.loc['1111_'+mrf+b2c, 'Направление B2C_верх'] = b2c.split('_')[0]+'_а'
        summary_itog.loc['1111_'+mrf+b2c, 'level'] = 1
        summary_itog.loc['1111_'+mrf+b2c,'Номер_назначения'] = b2c_up['Номер_назначения'].count()
        summary_itog.loc['1111_'+mrf+b2c, 'мес_оплат'] = b2c_up['мес_оплат'].mean()
        
        for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к', 'нет оплат 3 мес', 'кол_во']:
            summary_itog.loc['1111_'+mrf+b2c, cl] = b2c_up[cl].sum()
        summary_itog.loc['1111_'+mrf+b2c, 'Средняя_Выплата'] =  summary_itog.loc['1111_'+mrf+b2c, 'Всего']/summary_itog.loc['1111_'+mrf+b2c, 'мес_оплат']
        
        
        for b2cu in b2c_up['Направление B2C'].value_counts().index:
            b2c_all = b2c_up[b2c_up['Направление B2C'] == b2cu].copy()

            summary_itog.loc['1111_'+mrf+b2cu+b2c, 'МРФ'] = mrf
            summary_itog.loc['1111_'+mrf+b2cu+b2c, 'Направление B2C_верх'] = b2c
            summary_itog.loc['1111_'+mrf+b2cu+b2c, 'Направление B2C'] = b2cu.split('_')[0]+'_а'
            summary_itog.loc['1111_'+mrf+b2cu+b2c, 'level'] = 2
            summary_itog.loc['1111_'+mrf+b2cu+b2c,'Номер_назначения'] = b2c_all['Номер_назначения'].count()
            summary_itog.loc['1111_'+mrf+b2cu+b2c, 'мес_оплат'] = b2c_all['мес_оплат'].mean()
            
            for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к', 'нет оплат 3 мес', 'кол_во']:
                summary_itog.loc['1111_'+mrf+b2cu+b2c, cl] = b2c_all[cl].sum()
            summary_itog.loc['1111_'+mrf+b2cu+b2c, 'Средняя_Выплата'] =  summary_itog.loc['1111_'+mrf+b2cu+b2c, 'Всего']/summary_itog.loc['1111_'+mrf+b2c, 'мес_оплат']
            
            
            for fil in b2c_all['Филиала'].value_counts().index:
                fil_all = b2c_all[b2c_all['Филиала'] == fil].copy()
                summary_itog.loc['1111_'+mrf+fil+b2c+b2cu, 'МРФ'] = mrf
                summary_itog.loc['1111_'+mrf+fil+b2c+b2cu, 'Направление B2C_верх'] = b2c
                summary_itog.loc['1111_'+mrf+fil+b2c+b2cu, 'Направление B2C'] = b2cu
                summary_itog.loc['1111_'+mrf+fil+b2c+b2cu, 'Филиала'] = fil.split('_')[0]
                summary_itog.loc['1111_'+mrf+fil+b2c+b2cu, 'level'] = 3
                summary_itog.loc['1111_'+mrf+fil+b2c+b2cu,'Номер_назначения'] = fil_all['Номер_назначения'].count()
                summary_itog.loc['1111_'+mrf+fil+b2c+b2cu, 'мес_оплат'] = fil_all['мес_оплат'].mean()
                for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к', 'нет оплат 3 мес', 'кол_во']:
                    summary_itog.loc['1111_'+mrf+fil+b2c+b2cu, cl] = fil_all[cl].sum()
                summary_itog.loc['1111_'+mrf+fil+b2c+b2cu, 'Средняя_Выплата'] =  summary_itog.loc['1111_'+mrf+fil+b2c+b2cu, 'Всего']/summary_itog.loc['1111_'+mrf+fil+b2c+b2cu, 'мес_оплат']



CPU times: user 25.9 s, sys: 600 ms, total: 26.5 s
Wall time: 30.3 s


In [44]:
# сортировка


summary_itog.sort_values(['МРФ', 'Направление B2C_верх','Направление B2C','Филиала'],kind='mergesort', inplace= True)
summary_itog['МРФ'] = summary_itog['МРФ'].str.replace('1111_', '')
summary_itog['МРФ'] = summary_itog['МРФ'].str.replace('_а1', '')
summary_itog['Филиала'] = summary_itog['Филиала'].str.replace('_а1', '')
summary_itog['Направление B2C'] = summary_itog['Направление B2C'].str.replace('_а1', '')
summary_itog['Направление B2C_верх'] = summary_itog['Направление B2C_верх'].str.replace('_а1', '')
summary_itog['МРФ'] = summary_itog['МРФ'].str.replace('_а', '')
summary_itog['Филиала'] = summary_itog['Филиала'].str.replace('_а', '')
summary_itog['Направление B2C'] = summary_itog['Направление B2C'].str.replace('_а', '')
summary_itog['Направление B2C_верх'] = summary_itog['Направление B2C_верх'].str.replace('_а', '')

In [45]:
%%time

# выгрузка файла

summary_itog.reset_index(inplace=True, drop=True)
path = '/Users/ivansinazhenski/Documents/ WORK/ГПХ_B2C_Воронков_11_2020/масс_сектор_summary_v4.xlsx'
with pd.ExcelWriter(path, engine='xlsxwriter') as w:
    workbook = w.book
    summary_itog.to_excel(w, sheet_name='выплаты_все', index=False)
    out_sum_wsh = w.sheets['выплаты_все']
    out_sum_wsh.outline_settings(True, False, False, False)
    for i in summary_itog.index:
        if summary_itog.loc[i, 'level'] == 1:
            out_sum_wsh.set_row(i+1, None, None, {'level': 1})
        elif summary_itog.loc[i, 'level'] == 2:
            out_sum_wsh.set_row(i+1, None, None, {'level': 2})
        elif summary_itog.loc[i, 'level'] == 3:
            out_sum_wsh.set_row(i+1, None, None, {'level': 3})
        elif summary_itog.loc[i, 'level'] == 4:
            out_sum_wsh.set_row(i+1, None, None, {'level': 4})

CPU times: user 14.8 s, sys: 417 ms, total: 15.2 s
Wall time: 16.5 s


# Создание пивота только по направлениям B2C

In [ ]:
summary_b2c = summary.drop(['МРФ', 'Филиала'], axis=1).sort_values(['Направление B2C'])

In [ ]:
%%time

# общий итог
summary_itog = summary_b2c.copy()
summary_itog['Направление B2C'] = summary_itog['Направление B2C']+'_а1'
summary_itog['level'] = 2

summary_itog.loc['1111_Итог', 'Направление B2C'] = '1111_Итог'
summary_itog.loc['1111_Итог', 'level'] = '1'
summary_itog.loc['1111_Итог', 'Номер_назначения'] = summary_itog['Номер_назначения'].count()
for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к', 'нет оплат 3 мес']:
    summary_itog.loc['1111_Итог', cl] = summary_itog[cl].sum()

for mrf in summary_itog['Направление B2C'].value_counts().index:
    if mrf.find('1111_Итог') == 0:
        continue
    mrf_all = summary_itog[summary_itog['Направление B2C'] == mrf].copy()
    
    summary_itog.loc['1111_'+mrf, 'level'] = 1
    summary_itog.loc['1111_'+mrf, 'Направление B2C'] = mrf.split('_')[0]+'_а'
    summary_itog.loc['1111_'+mrf,'Номер_назначения'] = mrf_all['Номер_назначения' ].count()
    for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к','нет оплат 3 мес']:
        summary_itog.loc['1111_'+mrf, cl] = mrf_all[cl].sum()

In [ ]:
summary_itog.sort_values(['Направление B2C'], inplace= True)
summary_itog['Направление B2C'] = summary_itog['Направление B2C'].str.replace('1111_', '')
summary_itog['Направление B2C'] = summary_itog['Направление B2C'].str.replace('_а1', '')
summary_itog['Направление B2C'] = summary_itog['Направление B2C'].str.replace('_а', '')

In [ ]:
%%time

# выгрузка файла

summary_itog.reset_index(inplace=True, drop=True)
path = '/Users/ivansinazhenski/Documents/ WORK/ГПХ_B2C_Воронков_11_2020/масс_сектор_summary_b2c_v2.xlsx'
with pd.ExcelWriter(path, engine='xlsxwriter') as w:
    workbook = w.book
    summary_itog.to_excel(w, sheet_name='выплаты_b2c', index=False)
    out_sum_wsh = w.sheets['выплаты_b2c']
    out_sum_wsh.outline_settings(True, False, False, False)
    for i in summary_itog.index:
        if summary_itog.loc[i, 'level'] == 1:
            out_sum_wsh.set_row(i+1, None, None, {'level': 1})
        elif summary_itog.loc[i, 'level'] == 2:
            out_sum_wsh.set_row(i+1, None, None, {'level': 2})

# Превышение 500 

In [ ]:
summary_500 = summary[summary['Превышение >= 500к'] == 1].copy()

In [ ]:
%%time

# общий итог
summary_itog = summary_500.copy()
summary_itog['МРФ'] = summary_itog['МРФ']+'_а1'
summary_itog['Филиала'] = summary_itog['Филиала']+'_а1'
summary_itog['Направление B2C'] = summary_itog['Направление B2C']+'_а1'
summary_itog['level'] = 4

summary_itog.loc['1111_Итог', 'МРФ'] = '1111_Итог'
summary_itog.loc['1111_Итог', 'level'] = '1'
summary_itog.loc['1111_Итог', 'Номер_назначения'] = summary_itog['Номер_назначения'].count()
for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к', 'нет оплат 3 мес']:
    summary_itog.loc['1111_Итог', cl] = summary_itog[cl].sum()

for mrf in summary_itog.МРФ.value_counts().index:
    if mrf.find('1111_Итог') == 0:
        continue
    mrf_all = summary_itog[summary_itog['МРФ'] == mrf].copy()
    
    summary_itog.loc['1111_'+mrf, 'level'] = 1
    summary_itog.loc['1111_'+mrf, 'МРФ'] = mrf.split('_')[0]
    summary_itog.loc['1111_'+mrf,'Номер_назначения'] = mrf_all['Номер_назначения'].count()
    for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к', 'нет оплат 3 мес']:
        summary_itog.loc['1111_'+mrf, cl] = mrf_all[cl].sum()
        
    for b2c in mrf_all['Направление B2C'].value_counts().index:
        b2c_all = mrf_all[mrf_all['Направление B2C'] == b2c].copy()

        summary_itog.loc['1111_'+mrf+fil, 'МРФ'] = mrf
        summary_itog.loc['1111_'+mrf+fil, 'Направление B2C'] = b2c.split('_')[0]+'_а'
        summary_itog.loc['1111_'+mrf+fil, 'level'] = 2
        summary_itog.loc['1111_'+mrf+fil,'Номер_назначения'] = b2c_all['Номер_назначения'].count()
        for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к', 'нет оплат 3 мес']:
            summary_itog.loc['1111_'+mrf+fil, cl] = b2c_all[cl].sum()
    
        for fil in b2c_all['Филиала'].value_counts().index:
            fil_all = b2c_all[b2c_all['Филиала'] == fil].copy()
            summary_itog.loc['1111_'+mrf+fil+b2c, 'МРФ'] = mrf
            summary_itog.loc['1111_'+mrf+fil+b2c, 'Направление B2C'] = b2c
            summary_itog.loc['1111_'+mrf+fil+b2c, 'Филиала'] = fil.split('_')[0]
            summary_itog.loc['1111_'+mrf+fil+b2c, 'level'] = 3
            summary_itog.loc['1111_'+mrf+fil+b2c,'Номер_назначения'] = fil_all['Номер_назначения'].count()
            for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к', 'нет оплат 3 мес']:
                summary_itog.loc['1111_'+mrf+fil+b2c, cl] = fil_all[cl].sum()
            
     

In [ ]:
# сортировка

summary_itog.sort_values(['МРФ', 'Направление B2C','Филиала'], inplace= True)
summary_itog['МРФ'] = summary_itog['МРФ'].str.replace('1111_', '')
summary_itog['МРФ'] = summary_itog['МРФ'].str.replace('_а1', '')
summary_itog['Филиала'] = summary_itog['Филиала'].str.replace('_а1', '')
summary_itog['Направление B2C'] = summary_itog['Направление B2C'].str.replace('_а1', '')
summary_itog['МРФ'] = summary_itog['МРФ'].str.replace('_а', '')
summary_itog['Филиала'] = summary_itog['Филиала'].str.replace('_а', '')
summary_itog['Направление B2C'] = summary_itog['Направление B2C'].str.replace('_а', '')

In [ ]:
%%time

# выгрузка файла

summary_itog.reset_index(inplace=True, drop=True)
path = '/Users/ivansinazhenski/Documents/ WORK/ГПХ_B2C_Воронков_11_2020/масс_сектор_summary_500_v2.xlsx'
with pd.ExcelWriter(path, engine='xlsxwriter') as w:
    workbook = w.book
    summary_itog.to_excel(w, sheet_name='выплаты_500', index=False)
    out_sum_wsh = w.sheets['выплаты_500']
    out_sum_wsh.outline_settings(True, False, False, False)
    for i in summary_itog.index:
        if summary_itog.loc[i, 'level'] == 1:
            out_sum_wsh.set_row(i+1, None, None, {'level': 1})
        elif summary_itog.loc[i, 'level'] == 2:
            out_sum_wsh.set_row(i+1, None, None, {'level': 2})
        elif summary_itog.loc[i, 'level'] == 3:
            out_sum_wsh.set_row(i+1, None, None, {'level': 3})
        elif summary_itog.loc[i, 'level'] == 4:
            out_sum_wsh.set_row(i+1, None, None, {'level': 4})

# нет выплат 3 мес

In [ ]:
summary

In [ ]:
%%time

s_help = summary == 0
s_help = s_help.transpose()

for cl in s_help.columns:
    print(cl)
    l = s_help[s_help[cl] == True].index.to_list()
    summary.loc[cl, 'Месяца_без_оплат'] = str(l)

In [37]:
summary['Месяца_без_оплат'] = summary['Месяца_без_оплат'].astype(str)

In [38]:
summary.loc[summary[summary['Месяца_без_оплат'].str.contains('5, 6, 7')].index, 'нет оплат 3 мес'] = 1

In [39]:
summary['Месяца_без_оплат'] = summary['Месяца_без_оплат'].str.replace('1', 'янв')
summary['Месяца_без_оплат'] = summary['Месяца_без_оплат'].str.replace('2', 'февр')
summary['Месяца_без_оплат'] = summary['Месяца_без_оплат'].str.replace('3', 'март')
summary['Месяца_без_оплат'] = summary['Месяца_без_оплат'].str.replace('4', 'апр')
summary['Месяца_без_оплат'] = summary['Месяца_без_оплат'].str.replace('5', 'май')
summary['Месяца_без_оплат'] = summary['Месяца_без_оплат'].str.replace('6', 'июнь')
summary['Месяца_без_оплат'] = summary['Месяца_без_оплат'].str.replace('7', 'июль')
summary['Месяца_без_оплат'] = summary['Месяца_без_оплат'].str.replace('[','')
summary['Месяца_без_оплат'] = summary['Месяца_без_оплат'].str.replace(']','')

In [ ]:
summary_3mon_non = summary[summary['нет оплат 3 мес'] == 1].copy()

In [ ]:
%%time

# общий итог
summary_itog = summary_3mon_non .copy()
summary_itog['МРФ'] = summary_itog['МРФ']+'_а1'
summary_itog['Филиала'] = summary_itog['Филиала']+'_а1'
summary_itog['Направление B2C'] = summary_itog['Направление B2C']+'_а1'
summary_itog['level'] = 4

summary_itog.loc['1111_Итог', 'МРФ'] = '1111_Итог'
summary_itog.loc['1111_Итог', 'level'] = '1'
summary_itog.loc['1111_Итог', 'Номер_назначения'] = summary_itog['Номер_назначения'].count()
for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к', 'нет оплат 3 мес']:
    summary_itog.loc['1111_Итог', cl] = summary_itog[cl].sum()

for mrf in summary_itog.МРФ.value_counts().index:
    if mrf.find('1111_Итог') == 0:
        continue
    mrf_all = summary_itog[summary_itog['МРФ'] == mrf].copy()
    
    summary_itog.loc['1111_'+mrf, 'level'] = 1
    summary_itog.loc['1111_'+mrf, 'МРФ'] = mrf.split('_')[0]
    summary_itog.loc['1111_'+mrf,'Номер_назначения'] = mrf_all['Номер_назначения'].count()
    for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к', 'нет оплат 3 мес']:
        summary_itog.loc['1111_'+mrf, cl] = mrf_all[cl].sum()
        
    for b2c in mrf_all['Направление B2C'].value_counts().index:
        b2c_all = mrf_all[mrf_all['Направление B2C'] == b2c].copy()

        summary_itog.loc['1111_'+mrf+fil, 'МРФ'] = mrf
        summary_itog.loc['1111_'+mrf+fil, 'Направление B2C'] = b2c.split('_')[0]+'_а'
        summary_itog.loc['1111_'+mrf+fil, 'level'] = 2
        summary_itog.loc['1111_'+mrf+fil,'Номер_назначения'] = b2c_all['Номер_назначения'].count()
        for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к', 'нет оплат 3 мес']:
            summary_itog.loc['1111_'+mrf+fil, cl] = b2c_all[cl].sum()
    
        for fil in b2c_all['Филиала'].value_counts().index:
            fil_all = b2c_all[b2c_all['Филиала'] == fil].copy()
            summary_itog.loc['1111_'+mrf+fil+b2c, 'МРФ'] = mrf
            summary_itog.loc['1111_'+mrf+fil+b2c, 'Направление B2C'] = b2c
            summary_itog.loc['1111_'+mrf+fil+b2c, 'Филиала'] = fil.split('_')[0]
            summary_itog.loc['1111_'+mrf+fil+b2c, 'level'] = 3
            summary_itog.loc['1111_'+mrf+fil+b2c,'Номер_назначения'] = fil_all['Номер_назначения'].count()
            for cl in [1,2,3,4,5,6,7,'Всего','Превышение >= 500к', 'нет оплат 3 мес']:
                summary_itog.loc['1111_'+mrf+fil+b2c, cl] = fil_all[cl].sum()

In [ ]:
# сортировка

summary_itog.sort_values(['МРФ', 'Направление B2C','Филиала'], inplace= True)
summary_itog['МРФ'] = summary_itog['МРФ'].str.replace('1111_', '')
summary_itog['МРФ'] = summary_itog['МРФ'].str.replace('_а1', '')
summary_itog['Филиала'] = summary_itog['Филиала'].str.replace('_а1', '')
summary_itog['Направление B2C'] = summary_itog['Направление B2C'].str.replace('_а1', '')
summary_itog['МРФ'] = summary_itog['МРФ'].str.replace('_а', '')
summary_itog['Филиала'] = summary_itog['Филиала'].str.replace('_а', '')
summary_itog['Направление B2C'] = summary_itog['Направление B2C'].str.replace('_а', '')

In [ ]:
%%time

# выгрузка файла

summary_itog.reset_index(inplace=True, drop=True)
path = '/Users/ivansinazhenski/Documents/ WORK/ГПХ_B2C_Воронков_11_2020/масс_сектор_summary_3_v2.xlsx'
with pd.ExcelWriter(path, engine='xlsxwriter') as w:
    workbook = w.book
    summary_itog.to_excel(w, sheet_name='выплаты_3', index=False)
    out_sum_wsh = w.sheets['выплаты_3']
    out_sum_wsh.outline_settings(True, False, False, False)
    for i in summary_itog.index:
        if summary_itog.loc[i, 'level'] == 1:
            out_sum_wsh.set_row(i+1, None, None, {'level': 1})
        elif summary_itog.loc[i, 'level'] == 2:
            out_sum_wsh.set_row(i+1, None, None, {'level': 2})
        elif summary_itog.loc[i, 'level'] == 3:
            out_sum_wsh.set_row(i+1, None, None, {'level': 3})
        elif summary_itog.loc[i, 'level'] == 4:
            out_sum_wsh.set_row(i+1, None, None, {'level': 4})